# Оценивание параметров в Гауссовых смешанных моделях с помощью тензорных разложений

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [2]:
#d = 10, k = 6
d = 8 #размерность
k = 6  #число распределений
#k должно быть <= d из-за процедуры отбеливания
n = 100#количество точек
total = k * n  #точек всего
s = 5 #дисперсия
s_out = 3#дисперсия выброса
dist = 20
spher = True
cov_range = 2

In [3]:
true_labels = [0]*total

In [4]:
def generate_data():
    A = -dist+(dist+dist)*np.random.rand(d, k)
    X = np.zeros((total, d))

    for i in range(k):
        mean = np.transpose(A[:, i])
        if spher:
            covariance = s * np.identity(d)
        else:
            a = -cov_range + (cov_range + cov_range) * np.random.rand(d, d)
            covariance = np.matmul(np.transpose(a), a)
        mvn = np.random.multivariate_normal(mean, covariance, n)
        #plt.plot(mvn[:, 0], mvn[:, 1], '.')
        #plt.plot(mean[0], mean[1], 'x')
        X[i*n:(i+1)*n, :] = mvn
        for j in range(i*n, (i+1)*n):
            true_labels[j] = i
        
    return (X, A)

In [5]:
def calculate_first_moment(X):
    mu = np.zeros((d, 1))
    for t in range(total):
        for i in range(d):
            mu[i] += + X[t, i]
    mu /= total
    return mu

In [6]:
def calculate_second_moment(X):
    Sigma = np.zeros((d, d))
    for t in range(total):
        for i in range(d):
            for j in range(d):
                Sigma[i, j] += np.dot(X[t, i],X[t, j])
    Sigma /= total
    return Sigma

In [7]:
def extract_information_from_second_moment(Sigma, X):
    U, S, _ = np.linalg.svd(Sigma)
    s_est = S[-1]
    W, X_whit = perform_whitening(X, U, S)
    return (s_est, W, X_whit)

In [8]:
def perform_whitening(X, U, S):
    W = np.matmul(U[:, 0:k], np.sqrt(np.linalg.pinv(np.diag(S[0:k]))))
    X_whit = np.matmul(X, W)
    return (W, X_whit)

In [9]:
def perform_tensor_power_method(X_whit, W, s_est, mu):
    TOL = 1e-8
    maxiter = 100
    V_est = np.zeros((k, k))
    lamb = np.zeros((k, 1))

    for i in range(k):
        v_old = np.random.rand(k, 1)
        v_old = np.divide(v_old, np.linalg.norm(v_old))
        for iter in range(maxiter):
            v_new = (np.matmul(np.transpose(X_whit), (np.matmul(X_whit, v_old) * np.matmul(X_whit, v_old)))) / total
            #v_new = v_new - s_est * (W' * mu * dot((W*v_old),(W*v_old)));
            #v_new = v_new - s_est * (2 * W' * W * v_old * ((W'*mu)' * (v_old)));
            v_new -= s_est * (np.matmul(np.matmul(W.T, mu), np.dot(np.matmul(W, v_old).T,np.matmul(W, v_old))))
            v_new -= s_est * (2 * np.matmul(W.T, np.matmul(W, np.matmul(v_old, np.matmul(np.matmul(W.T, mu).T, v_old)))))
            if i > 0:
                for j in range(i):
                    v_new -= np.reshape(V_est[:, j] * np.power(np.matmul(np.transpose(v_old), V_est[:, j]), 2) * lamb[j], (k, 1))
            l = np.linalg.norm(v_new)
            v_new = np.divide(v_new, np.linalg.norm(v_new))
            if np.linalg.norm(v_old - v_new) < TOL:
                V_est[:, i] = np.reshape(v_new, k)
                lamb[i] = l
                break
            v_old = v_new
    
    return (V_est, lamb)

In [10]:
def perform_backwards_transformation(V_est, lamb):
    return np.matmul(np.matmul(np.linalg.pinv(np.transpose(W)), V_est), np.diag(np.reshape(lamb.T, k)))

In [11]:
import scipy
def find_mode(lst):
    return stats.mode(lst)
def multivariateGaussian(X, mu, sigma):
    k = len(mu)
    X = X - mu.T
    p = 1/((2*np.pi)**(k/2)*(np.linalg.det(sigma)**0.5))* np.exp(-0.5* np.sum(X @ np.linalg.pinv(sigma) * X,axis=1))
    return p


In [12]:
def Kmeans(X):
    dim = [0]*k
    for i in range(k):
        dim[i] = list()

    for j in range(k):
        for i in range(len(X)):
            dim[j].append(X[i][j])


    Data = {'x': dim[0],
            'y': dim[1]
           }

    df = DataFrame(Data,columns=['x','y'])

    kmeans = KMeans(n_clusters=k).fit(df)

    kmeans_errors = 0
    for i in range(k):
        mode = scipy.stats.mode(kmeans.labels_[i*n:(i+1)*n])
        for j in range(i*n,(i+1)*n):
            if kmeans.labels_[j] != int(mode[0]):
                kmeans_errors += 1
    return kmeans_errors

In [13]:
def Gmm(X):
    dim = [0]*k
    for i in range(k):
        dim[i] = list()

    for j in range(k):
        for i in range(len(X)):
            dim[j].append(X[i][j])


    Data = {'x': dim[0],
            'y': dim[1]
           }

    df = DataFrame(Data,columns=['x','y'])

    gmm = GaussianMixture(n_components=k)
    gmm.fit(df)

    
    gmm_labels = gmm.predict(df)

    gmm_errors = 0
    for i in range(k):
        mode = scipy.stats.mode(gmm_labels[i*n:(i+1)*n])
        for j in range(i*n,(i+1)*n):
            if gmm_labels[j] != int(mode[0]):
                gmm_errors += 1
    return gmm_errors

In [14]:
def MyAlgorithm(X,A_est,Sigma):
    distances_from_center = [0] * k

    estimated_means = []
    for i in range(k):
        mean_est = A_est[:, i].T
        estimated_means.append(mean_est)

    my_algorithm_clusters =[]

    for j in range(k):
        p = multivariateGaussian(X,estimated_means[j],Sigma)
        cluster_x = list()
        cluster_y = list()

        list1 = p
        list2 = X
        list1, list2 = zip(*sorted(zip(list1, list2), reverse=True))
        for i in list2[:n]:
            dist = (i[0] - estimated_means[j][0])**2 + (i[1] - estimated_means[j][1])**2
            if (dist < 9*(s_est**2)):
                distances_from_center[j] += dist
                cluster_x.append(i[0])
                cluster_y.append(i[1])
                my_algorithm_clusters.append(j)

    errors = 0
    for i in range(len(my_algorithm_clusters)):
        if (my_algorithm_clusters[i] != true_labels[i]):
            errors += 1
    return errors

In [15]:
TOTAL_ITERATIONS = 1000
my_alg_average = 0
gmm_average = 0
kmeans_average = 0
for t in range(TOTAL_ITERATIONS):
    X, A = generate_data()

    mu = calculate_first_moment(X)
    Sigma = calculate_second_moment(X)

    s_est, W, X_whit = extract_information_from_second_moment(Sigma, X)

    V_est, lamb = perform_tensor_power_method(X_whit, W, s_est, mu)

    A_est = perform_backwards_transformation(V_est, lamb)
    
    kmeans_errors = Kmeans(X)
    gmm_errors = Gmm(X)
    my_errors = MyAlgorithm(X,A_est,Sigma)
    
    My_algorithm_precision = (1-my_errors/len(true_labels))*100
    my_alg_average += My_algorithm_precision

    gmm_precision = (1-gmm_errors/len(true_labels))*100
    gmm_average += gmm_precision
    
    kmeans_precision = (1-kmeans_errors/len(true_labels))*100
    kmeans_average += kmeans_precision
    
kmeans_average /= TOTAL_ITERATIONS
gmm_average /= TOTAL_ITERATIONS
my_alg_average /= TOTAL_ITERATIONS

print("Экспериментов было проведено", TOTAL_ITERATIONS)
print("Средняя точность Kmeans",kmeans_average,"%")
print("Средняя точность GMM",gmm_average,"%")
print("Средняя точность моего алгоритма",my_alg_average,"%")

Экспериментов было проведено 1000
Средняя точность Kmeans 91.92816666666651 %
Средняя точность GMM 91.76233333333325 %
Средняя точность моего алгоритма 94.86599999999994 %
